In [1]:
from dotenv import load_dotenv
import os

# .env 불러오기
load_dotenv()

# 기본 모델 설정
# 여기서 설정할 수도 있지만 .env 내에 넣어줘도 됨
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [2]:
from pydantic import BaseModel
from typing import List


# pydantic을 통해 outputs의 형식을 강제할 수 있음
class SubSection(BaseModel):
    title: str
    content: str


class BlogPost(BaseModel):
    title: str
    introduction: str
    sections: List[SubSection]
    sources: List[str]
    hashtags: List[str]

In [3]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

# Tools 설정
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information.",
    # You want to impress your coworkers with your work.
    # 위와 같은 문장을 통해 더 좋은 답변을 받을 수 있음
    # 누군가에게 감명을 준다거나 중요한 고객을 위해 일한다거나 하는 등 부연설명을 붙여주면 좋음
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up to date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,  # 다른 agent와 협업을 하지 않고 혼자 일하도록 강제하는 것
    verbose=True,
    # 사용할 Tools 명시
    tools=[
        # 구글 검색 후 링크를 가져옴
        search_tool,
        # 웹사이트에 들어가 내용을 가져옴
        scrape_tool,
    ],
    # 과도한 과금이 발생하지 않도록 조사 횟수를 제한할 수 있음
    # 제한할 수 있는 방법에는 2가지가 있음
    # 1. max_excution_time: 최대 실행 시간을 명시하여 제한
    # 2. max_iter: 답변을 출력하기 전 최대한 작업을 반복할 수 있는 횟수
    max_iter=10,
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts.",
    backstory="""
    You write content that keeps people engaged and entertained.
    Your content is easy to read it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
    # editor는 delegation을 허용함으로써 컨텥트가 없다면 다른 agent에게 자료를 찾아오도록 해야 함
    allow_delegation=True,
)

task = Task(
    description="Write a blog post about {topic}",
    # Task를 수행할 agent 명시
    agent=editor,
    expected_output="A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    # 다음과 같이 작성 시 pydantic을 만족하는 결과물이 생성되고 그 결과가 blog_post.md 파일에 저장됨
    output_file="blog_post.md",
    output_pydantic=BlogPost,
    # 다음과 같이 작성 시 json 형식으로 바로 반환
    # output_json=BlogPost,
)

crew = Crew(
    # 사용할 모든 agents와 tasks를 명시
    agents=[
        researcher,
        editor,
    ],
    tasks=[task],
    verbose=True,
)

result = crew.kickoff(
    inputs=dict(
        # flops: 기대보다 흥행에 실패한 영화
        topic="The biggest box office flops of 2024",
    ),
)

# Agent: Senior Writer/Editor
## Task: Write a blog post about The biggest box office flops of 2024
 

I encountered an error while trying to use the tool. This was the error: Arguments validation failed: 2 validation errors for DelegateWorkToolSchema
task
  Input should be a valid string [type=string_type, input_value={'description': 'Research...ontext.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type
context
  Input should be a valid string [type=string_type, input_value={'description': 'This blo...ovided.', 'type': 'str'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type.
 Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'description': 'The context for the task', 'type': 'str'}, 'coworker': {'description': 'The role/name of the coworker 

In [6]:
# CrewAI 버전(0.41.1)이 더 높다면 result.pydantic.sections, result.pydantic.hashtags 형태로 접근 가능합니다.
result.pydantic.sources

['https://www.boxofficemojo.com/',
 'https://www.rottentomatoes.com/',
 'https://www.indiewire.com/']

In [7]:
result.pydantic.title

"2024's Biggest Box Office Blunders: A Look at the Flops That Missed the Mark"